In [1]:
from esa import SAW #To install this do: python -m pip install esa
import pandas as pd
import numpy as np
##Change CPATH to the full path of case you are using
CPATH = r'C:\Users\maxbr\OneDrive - Texas A&M University\Grad\PowerWorld\ACTIVSg2000\ACTIVSg2000.PWB'

saw = SAW(CPATH)

In [2]:
UFLS_Key_fields = saw.get_key_field_list('LoadRelay_DLSH')
'''Frequency:X is the Frequency Load Shedding Point
   TSFrac:X is the Fraction of Load to Shed at its respective frequency point
   TST:X is the Pickup Time at its respective frequency point
   TSTb:X is the Breaker Time at its respective frequency point'''
   
otherfields = ['Frequency:1', 'TSFrac:1', 'TST:1','Frequency:2', 'TSFrac:2', 'TST:2', 'Frequency:3', 'TSFrac:3', 'TST:3', 'TSTb', 'FrequencyRate:1', 'FrequencyRate:2', 'FrequencyRate:3']
UFLS_Data = saw.GetParametersMultipleElement('LoadRelay_DLSH', UFLS_Key_fields + otherfields)

In [3]:
kf = saw.get_key_field_list('Load')
loads = saw.GetParametersMultipleElement('Load', kf )
loads

,BusNum,LoadID
0,1001,1
1,1002,1
2,1007,1
3,1012,1
4,1013,1
...,...,...
1345,8151,1
1346,8152,1
1347,8153,1
1348,8154,1


In [4]:
if UFLS_Data is None: #If there are no relays of this type create the data frame
    UFLS_Data = pd.DataFrame(columns=UFLS_Key_fields + otherfields)
else: ##Clears any existing relays in the system and just keeps the columns
    UFLS_Data = UFLS_Data[0:0].copy(deep=True) 

In [5]:
load_kf = saw.get_key_field_list('load') 
load_data = saw.GetParametersMultipleElement('load', load_kf) ##Gets every load in the system

In [6]:
UFLS_Data = pd.concat([UFLS_Data, load_data], ignore_index=True) ## Adds the load data to dataframe
UFLS_Data

,ObjectType,BusNum,LoadID,Frequency:1,TSFrac:1,TST:1,Frequency:2,TSFrac:2,TST:2,Frequency:3,TSFrac:3,TST:3,TSTb,FrequencyRate:1,FrequencyRate:2,FrequencyRate:3
0,NaN,1001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1002,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1007,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1013,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,NaN,8151,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1346,NaN,8152,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1347,NaN,8153,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1348,NaN,8154,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# ######0.1 for generators 0.05 for high voltage ##########
UFLS_Data.loc[:, 'ObjectType'] = 'Load' ##This adds Load to every row under column ObjectType
UFLS_Data.loc[:, 'Frequency:1'] = 59.3
UFLS_Data.loc[:, 'Frequency:2'] = 58.9
UFLS_Data.loc[:, 'Frequency:3'] = 58.5

UFLS_Data.loc[:, 'TSFrac:1'] = 0.05
UFLS_Data.loc[:, 'TSFrac:2'] = 0.1
UFLS_Data.loc[:, 'TSFrac:3'] = 0.1

#If there are multiple columns that will have the same value you can do the following
UFLS_Data.loc[:, 'TST:1'] = UFLS_Data.loc[:, 'TST:2'] = UFLS_Data.loc[:, 'TST:3'] = 0.2

UFLS_Data.loc[:, 'FrequencyRate:1'] = UFLS_Data.loc[:, 'FrequencyRate:2'] = UFLS_Data.loc[:, 'FrequencyRate:3'] = 0

UFLS_Data.loc[:, 'TSTb'] = 0.05


In [8]:
#This pushes the dataframe into powerworld, if theres any issues with how you created the dataframe, there will be a terminal error
saw.change_and_confirm_params_multiple_element(ObjectType='LoadRelay_DLSH', command_df=UFLS_Data)
#Finally Check your results
new_UFLS = saw.GetParametersMultipleElement('LoadRelay_DLSH', UFLS_Key_fields + otherfields)
new_UFLS


,ObjectType,BusNum,LoadID,Frequency:1,TSFrac:1,TST:1,Frequency:2,TSFrac:2,TST:2,Frequency:3,TSFrac:3,TST:3,TSTb,FrequencyRate:1,FrequencyRate:2,FrequencyRate:3
0,Load,1001,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
1,Load,1002,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
2,Load,1007,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
3,Load,1012,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
4,Load,1013,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Load,8151,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
1346,Load,8152,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
1347,Load,8153,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0
1348,Load,8154,1,59.3,0.05,0.2,58.9,0.15,0.2,58.5,0.25,0.2,0.05,0,0,0


In [9]:
#Once you know that everything looks right, save the case. 
saw.WriteAuxFile("UFLS_protection.aux", "","LoadRelay_DLSH", "ALL" )
saw.exit() ###Always include this after you finish using powerworld.